In [12]:
import os
import csv
import pandas as pd
import numpy as np
import skfeature.function
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [2]:
data = pd.read_csv("../feature_selection.csv")

In [3]:
data = data.drop("Unnamed: 0", axis = 1)

In [4]:
data.columns

Index(['SEQN', 'CBD120', 'CBD130', 'DPQ010', 'DPQ020', 'DPQ030', 'DPQ040',
       'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090', 'DUQ211', 'DUQ240',
       'FSQ012', 'FSQ162', 'HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031D',
       'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'OCQ180', 'OCD270',
       'PAD645', 'PAQ655', 'PAD660', 'PAQ665', 'SLD010H', 'SMQ040', 'SMD460',
       'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDEDUC3', 'DMDMARTL', 'RIDEXPRG',
       'DMDHHSIZ', 'INDHHIN2', 'INDFMPIR', 'DRQSDIET', 'DRQSDT1', 'BMXWT',
       'BMXBMI', 'SLQ050'],
      dtype='object')

In [5]:
X = data.drop("SLQ050", axis = 1)
y = data["SLQ050"]

In [6]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [7]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------


------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0


Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------


...

Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1


Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0


Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------


------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 2.0
Encoded Label: 1
------------
Original Class: 1.0
Encoded Label: 0


In [8]:
one_hot_y = to_categorical(encoded_y)


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [24]:
Xcat=X[['DPQ010', 'DPQ020', 'DPQ030', 'DPQ040',
       'DPQ050', 'DPQ060', 'DPQ070', 'DPQ080', 'DPQ090','DUQ211', 'DUQ240', 'FSQ012', 'FSQ162','HIQ011', 'HIQ031A', 'HIQ031B', 'HIQ031D',
       'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I','SMQ040','SMD460','RIAGENDR','RIDRETH1','DMDEDUC3', 'DMDMARTL','RIDEXPRG','INDHHIN2','DRQSDIET']]

In [25]:
Xcontig =X[['SEQN','CBD120','CBD130','PAD645', 'PAQ655', 'PAD660', 'PAQ665', 'SLD010H','OCQ180', 'OCD270', 'PAD645', 'PAQ655', 'PAD660', 'PAQ665','SLD010H','RIDAGEYR','DMDHHSIZ','INDFMPIR','BMXWT',
       'BMXBMI']]

IndexError: index -2147483648 is out of bounds for axis 1 with size 1000